## 1. Construct the emission matrix p(w/t)
In the trigram model, I label the word using the tag consists of its own tag with the previous tag. For example, if there are 3 different tags, A, B, C. Now the tag list for the trigram model is [start, startA,AB,BA,AC,CA,BC,CB,AA,BB,CC,stop].  

In [1]:
## Construct bag of word for word and tag seperately
word_count = {}
tag_count = {}
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        temp_tag_list = []
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            if temp[0] not in word_count:
                word_count[temp[0]]=1
            if temp[0] in word_count:
                word_count[temp[0]]+=1
            temp_tag_list.append(temp[1])
        ## Construct bigram tags
        temp_tag_list_prev = ["<s>"]
        for i in range(len(temp_tag_list)-1):
            temp_tag_list_prev.append(temp_tag_list[i])
        for i in range(len(temp_tag_list)):
            tag_temp= "".join([temp_tag_list_prev[i],temp_tag_list[i]])
            if tag_temp not in tag_count:
                tag_count[tag_temp] = 1
            else:
                tag_count[tag_temp] += 1
### word_count
#{'azaleas': 5,
# 'termination': 3,
# 'planting': 26,
# ...

### tag_count
#{'<s>A': 28,
# '<s>D': 11005,
# '<s>E': 1978,
# '<s>F': 12612,
# '<s>N': 153233,
# '<s>R': 11318,
# 'AA': 2564,
# 'AD': 19,
# 'AE': 2,
# ... 

In [32]:
print (tag_count)

{'EE': 5354, 'ND': 11910, 'DA': 38, 'AF': 46, 'EA': 1, 'FN': 27764, 'NN': 1101003, 'NE': 653, '<s>E': 1978, 'DF': 736, '<s>R': 11318, 'AD': 19, 'AA': 2564, 'FA': 89, '<s>D': 11005, 'FD': 1999, 'NF': 20663, '<s>F': 12612, 'RE': 440, 'RD': 2172, 'EF': 64, '<s>N': 153233, 'RR': 63028, 'NR': 21839, 'NA': 174, '<s>A': 28, 'DD': 38134, 'DE': 114, 'AE': 2, 'RA': 136, 'DR': 2685, 'ED': 142, 'DN': 18923, 'FE': 83, 'AR': 48, 'EN': 2378, 'FF': 39789, 'ER': 499, 'RN': 30003, 'FR': 2317, 'RF': 4145, 'AN': 305}


In [2]:
## Construct the data structure to store the c(t,w). Here t is the double tag created above.  
# Use the dictionary to store the counts
# For the dictionary word_tag_pair_counts, the keys are the tags in the training set, the value is a dictionary, in which the keys 
# are the words, the the value are the counts for this word-key pair 
tag_word_pair_counts = {}

with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        
        temp_tag_list_uni = []
        temp_tag_list_double = []
        temp_word_list = []
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            temp_word_list.append(temp[0])
            temp_tag_list_uni.append(temp[1])
        
        ## Construct the double tag    
        temp_tag_list_uni_prev = ["<s>"]
        for i in range(len(temp_tag_list_uni)-1):
            temp_tag_list_uni_prev.append(temp_tag_list_uni[i])
        for i in range(len(temp_tag_list_uni)):
            temp_tag_list_double.append("".join([temp_tag_list_uni_prev[i],temp_tag_list_uni[i]]))
        
        for i in range(len(temp_tag_list_double)):
            if temp_tag_list_double[i] not in tag_word_pair_counts:
                tag_word_pair_counts[temp_tag_list_double[i]] = {}
                tag_word_pair_counts[temp_tag_list_double[i]][temp_word_list[i]] = 1
            else:
                if temp_word_list[i] not in tag_word_pair_counts[temp_tag_list_double[i]]:
                    tag_word_pair_counts[temp_tag_list_double[i]][temp_word_list[i]] = 1
                else:
                    tag_word_pair_counts[temp_tag_list_double[i]][temp_word_list[i]] += 1
#tag_word_pair_counts
#{'<s>A': {'--': 8,
#  'and': 1,
#  'basketball': 1,
#  'but': 1,
#  'criminal': 1,
#  'frankly': 1,
#  'i': 2,
#  'like': 1,
#  'not': 3,
# ...
# ...

In [3]:
## Construct the emission matrix, each entry is p(w/t), I use add delta smoothing to deal with unknown word

## I use a dictionary of dictionary to represent this matrix
## The key of the outer dictionary is the double tag, and the the key of the sub-dictionary is their corresponding word
emission_matrix = {}
delta = 0.1 
V = len(word_count) + 1 ## assum only one kind of unseen words
for tag in tag_count:
    emission_matrix[tag]={}
    sum_tag_allword = 0
    for word in tag_word_pair_counts[tag]:
        sum_tag_allword += tag_word_pair_counts[tag][word]
    sum_tag_allword += V*delta ## Add delta smoothing
    for word in tag_word_pair_counts[tag]:
        emission_matrix[tag][word] = (tag_word_pair_counts[tag][word]+delta)/sum_tag_allword
    emission_matrix[tag]['unk'] = delta/sum_tag_allword
#emission_matrix
#{'<s>A': {'--': 0.0039187227866473145,
#  'and': 0.0005321722302854379,
#  'basketball': 0.0005321722302854379,
#  'but': 0.0005321722302854379,
#  'criminal': 0.0005321722302854379,
#  'frankly': 0.0005321722302854379,
# ...

In [26]:
## Obtain p(you/t)
count = 0
for tag in tag_count:    
    if "you" not in emission_matrix[tag]:
        continue
    else:
        count += 1 
        print ("p(you/{}): {}".format(tag,emission_matrix[tag]['you']))
print (count)

p(you/ND): 0.6603412430998639
p(you/FN): 0.011478710196960038
p(you/NN): 0.012527265507569068
p(you/NE): 0.0018945022288261513
p(you/<s>E): 0.0002738361961662933
p(you/<s>R): 0.01498090888672606
p(you/AD): 0.005393586005830904
p(you/AA): 0.0037149684988051273
p(you/FA): 0.0005169172932330828
p(you/<s>D): 0.15440815700705304
p(you/FD): 0.27144626052501236
p(you/<s>F): 7.50801993038018e-05
p(you/RE): 0.0004437273093989512
p(you/RD): 0.3818807884113037
p(you/<s>N): 0.012243675614405687
p(you/RR): 0.009069113375443773
p(you/NR): 0.015876539073624258
p(you/NA): 0.002756439222774514
p(you/DD): 0.008167176959649516
p(you/RA): 0.0018850574712643677
p(you/DR): 0.017379339542760372
p(you/ED): 0.05048143053645117
p(you/DN): 0.026242724930827212
p(you/FE): 0.0005183788878416588
p(you/EN): 0.02583201267828843
p(you/ER): 0.007919621749408984
p(you/RN): 0.017230509955683166
p(you/FR): 0.010583103764921947
p(you/AN): 0.0013225255972696246
29


## 2. Construct transtion matrix p(t'/t) 

In [4]:
## Considering the start and stop tag, there are 44 different tags 
## t =  [<s>,'AD', 'DA'...]
## t' = ['AD', 'DA', </s>...]

## Construct a matrix to store the counts of c(t',t) 
## I use a dictionary of dictionary to represent this matrix
## The key of the outer dictionary is t, and the the key of the sub-dictionary is t'

tag_tag_count = {}
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        
        temp_tag_list_uni = []
        

        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            temp_tag_list_uni.append(temp[1])
            
        ## Construct the double tag    
        temp_tag_list_uni_prev = ["<s>"]
        for i in range(len(temp_tag_list_uni)-1):
            temp_tag_list_uni_prev.append(temp_tag_list_uni[i])
            
        temp_tag_list_double = ["<s>"]
        
        for i in range(len(temp_tag_list_uni)):
            temp_tag_list_double.append("".join([temp_tag_list_uni_prev[i],temp_tag_list_uni[i]]))
        
        temp_tag_list_double.append("</s>")
        
        
        for i in range(len(temp_tag_list_double)-1): ## i go through all the t in one sentence
            if temp_tag_list_double[i] not in tag_tag_count:
                tag_tag_count[temp_tag_list_double[i]]={}
                tag_tag_count[temp_tag_list_double[i]][temp_tag_list_double[i+1]]=1
            else:
                if temp_tag_list_double[i+1] not in tag_tag_count[temp_tag_list_double[i]]:
                    tag_tag_count[temp_tag_list_double[i]][temp_tag_list_double[i+1]]=1
                else:
                    tag_tag_count[temp_tag_list_double[i]][temp_tag_list_double[i+1]]+=1
                    
    

In [28]:
tag_tag_count

{'<s>': {'<s>A': 28,
  '<s>D': 11005,
  '<s>E': 1978,
  '<s>F': 12612,
  '<s>N': 153233,
  '<s>R': 11318},
 '<s>A': {'AA': 26, 'AD': 1, 'AN': 1},
 '<s>D': {'</s>': 29, 'DD': 8991, 'DE': 4, 'DF': 6, 'DN': 1688, 'DR': 287},
 '<s>E': {'</s>': 1, 'EE': 1962, 'EN': 12, 'ER': 3},
 '<s>F': {'</s>': 38, 'FD': 18, 'FF': 11916, 'FN': 619, 'FR': 21},
 '<s>N': {'</s>': 1934,
  'NA': 6,
  'ND': 864,
  'NE': 155,
  'NF': 526,
  'NN': 145678,
  'NR': 4070},
 '<s>R': {'</s>': 14, 'RD': 25, 'RE': 6, 'RF': 24, 'RN': 259, 'RR': 10990},
 'AA': {'</s>': 45,
  'AA': 2107,
  'AD': 18,
  'AE': 2,
  'AF': 46,
  'AN': 304,
  'AR': 42},
 'AD': {'DA': 2, 'DD': 15, 'DN': 2},
 'AE': {'EE': 2},
 'AF': {'FA': 1, 'FF': 43, 'FN': 1, 'FR': 1},
 'AN': {'</s>': 20, 'NN': 277, 'NR': 8},
 'AR': {'RN': 3, 'RR': 45},
 'DA': {'AA': 37, 'AR': 1},
 'DD': {'</s>': 4676,
  'DA': 31,
  'DD': 14869,
  'DE': 106,
  'DF': 729,
  'DN': 15479,
  'DR': 2244},
 'DE': {'EE': 113, 'EN': 1},
 'DF': {'</s>': 1, 'FD': 1, 'FF': 715, 'FN': 19},


In [5]:
## Construct transition matrix
## Construct the transition matrix to store the p(t'/t). The data structure is the same as that store c(t',t)
transition_matrix = {}
for tag in tag_tag_count:
    transition_matrix[tag] = {}
    sum_temp = 0
    for taggg in tag_tag_count[tag]:
        sum_temp += tag_tag_count[tag][taggg]
    for taggg in tag_tag_count[tag]:
        transition_matrix[tag][taggg] = tag_tag_count[tag][taggg]/sum_temp

In [30]:
transition_matrix

{'<s>': {'<s>A': 0.0001472335860843228,
  '<s>D': 0.05786805767349901,
  '<s>E': 0.010401001188385373,
  '<s>F': 0.0663182138462671,
  '<s>N': 0.8057515748735369,
  '<s>R': 0.05951391883222733},
 '<s>A': {'AA': 0.9285714285714286,
  'AD': 0.03571428571428571,
  'AN': 0.03571428571428571},
 '<s>D': {'</s>': 0.002635165833711949,
  'DD': 0.8169922762380736,
  'DE': 0.0003634711494775102,
  'DF': 0.0005452067242162653,
  'DN': 0.15338482507950932,
  'DR': 0.026079054975011357},
 '<s>E': {'</s>': 0.0005055611729019212,
  'EE': 0.9919110212335692,
  'EN': 0.006066734074823054,
  'ER': 0.0015166835187057635},
 '<s>F': {'</s>': 0.0030130034887408817,
  'FD': 0.0014272121788772598,
  'FF': 0.9448144624167459,
  'FN': 0.049080241040279096,
  'FR': 0.001665080875356803},
 '<s>N': {'</s>': 0.012621302199917772,
  'NA': 3.915605646303342e-05,
  'ND': 0.0056384721306768125,
  'NE': 0.0010115314586283633,
  'NF': 0.0034326809499259296,
  'NN': 0.9506959989036304,
  'NR': 0.02656085830075767},
 '<s>R

## 3. Decoding on the test file -- Viterbi Algorithm

In [6]:
import numpy as np

In [37]:
## Viterbi Algorithm Implementation
## Dynamic Programming
tag_list = ['<s>','EE', 'ND', 'DA', 'AF', 'EA', 'FN', 'NN', 'NE', '<s>E', 'DF', '<s>R', 'AD', 'AA', 'FA', '<s>D', 'FD', 'NF', '<s>F','RE','RD','EF','<s>N','RR','NR','NA','<s>A','DD','DE','AE','RA','DR','ED','DN','FE','AR','EN','FF','ER','RN','FR','RF','AN','</s>']

count_correct_prediction = 0
count_totalword = 0
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/test.txt") as test_file:
    for line in test_file:
        word_eachline = []
        tag_eachline = []
        tag_eachline.append("<s>")
        line_list = line.split()
        for item in line_list:
            temp = item.split('/')
            word_eachline.append(temp[0])
            tag_eachline.append(temp[1])
        tag_eachline.append('</s>')
        ## Use Viterbi Algorithm
        ## Viterbi matrix is len(tag_list)*len(word_eachline)
        viterbi = np.zeros((len(tag_list),len(word_eachline))) ## intialize the viterbi matrx all zeros 
        backpointer = np.zeros((len(tag_list),len(word_eachline)),dtype=int) ## intialize the backpointer matrx all zeros
        for i in range(1, len(tag_list)-1): ## iterate all tag without start and stop tags
            ## Get the entries for the first column in Viterbi matrix
            if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
                viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]][word_eachline[0]]
            
            if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
                viterbi[i,0] = 0
            
            if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
                viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]]["unk"]
                
            if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
                viterbi[i,0] = 0
                
            backpointer[i,0] = 0 ## 0 means the previous tag is '<s>'
            
        ## Calculate other entries
        for i in range(1,len(word_eachline)):
            for j in range(1, len(tag_list)-1):
                temp1 = []
                temp2 = []
                for k in range(1, len(tag_list)-1):
                    if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                        temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]][word_eachline[i]])
                        temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                    if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                        temp1.append(0)
                        temp2.append(0)
                    if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                        temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]]['unk'])
                        temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                    if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                        temp1.append(0)
                        temp2.append(0)
                viterbi[j,i] = max(temp1)
                backpointer[j,i]= temp2.index(max(temp2))+1  ## here if there are more than one largest values in temp2, the index function only get the smallest index
                
        ## Temination step
        temp = []
        for i in range(1, len(tag_list)-1):
            if "</s>" in transition_matrix[tag_list[i]]:
                temp.append(viterbi[i,-1]*transition_matrix[tag_list[i]]['</s>'])
            else:
                temp.append(0)
        viterbi[tag_list.index("</s>"),-1] = max(temp)
        backpointer[tag_list.index("</s>"),-1] = temp.index(max(temp))+1
        
        ## Backtrack to get the tag sequence
        predict_eachline = ["</s>"]
        predict_eachline.append(tag_list[backpointer[-1,-1]])
        prev = backpointer[-1,-1]

        for i in range(len(word_eachline)-1, 0, -1):
            predict_eachline.append(tag_list[backpointer[prev,i]])
            prev = backpointer[prev,i]
        predict_eachline.append(tag_list[backpointer[prev,0]])
        predict_eachline = predict_eachline[::-1]
        
        predict_eachline_transform_back_unitag = ["<s>"]
        
        for i in range(1,len(predict_eachline)-1):
            predict_eachline_transform_back_unitag.append(predict_eachline[i][-1])
        
        predict_eachline_transform_back_unitag.append('</s>')
            
        
        ## Count the correct prediction, not count the start and stop tags
        for i in range(1,len(predict_eachline_transform_back_unitag)-1):
            count_totalword += 1
            if predict_eachline_transform_back_unitag[i] == tag_eachline[i]:
                count_correct_prediction += 1    

In [39]:
print ("The prediction accuracy by trigram hidden Markov model is : {}.".format(count_correct_prediction/count_totalword))

The prediction accuracy by trigram hidden Markov model is : 0.920294416944066.


## 4. Prediction on the second line of text file

In [9]:
tag_list = ['<s>','EE', 'ND', 'DA', 'AF', 'EA', 'FN', 'NN', 'NE', '<s>E', 'DF', '<s>R', 'AD', 'AA', 'FA', '<s>D', 'FD', 'NF', '<s>F','RE','RD','EF','<s>N','RR','NR','NA','<s>A','DD','DE','AE','RA','DR','ED','DN','FE','AR','EN','FF','ER','RN','FR','RF','AN','</s>']

count_correct_prediction = 0
count_totalword = 0
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/test.txt") as test_file:
    first_line = next(test_file)
    second_line = next(test_file)
    word_eachline = []
    tag_eachline = []
    tag_eachline.append("<s>")
    line_list = second_line.split()
    for item in line_list:
        temp = item.split('/')
        word_eachline.append(temp[0])
        tag_eachline.append(temp[1])
    tag_eachline.append('</s>')
    ## Use Viterbi Algorithm
    ## Viterbi matrix is len(tag_list)*len(word_eachline)
    viterbi = np.zeros((len(tag_list),len(word_eachline))) ## intialize the viterbi matrx all zeros 
    backpointer = np.zeros((len(tag_list),len(word_eachline)),dtype=int) ## intialize the backpointer matrx all zeros
    for i in range(1, len(tag_list)-1): ## iterate all tag without start and stop tags
        ## Get the entries for the first column in Viterbi matrix
        if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
            viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]][word_eachline[0]]
            
        if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
            viterbi[i,0] = 0
            
        if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
            viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]]["unk"]
                
        if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
            viterbi[i,0] = 0
                
        backpointer[i,0] = 0 ## 0 means the previous tag is '<s>'
            
    ## Calculate other entries
    for i in range(1,len(word_eachline)):
        for j in range(1, len(tag_list)-1):
            temp1 = []
            temp2 = []
            for k in range(1, len(tag_list)-1):
                if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                    temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]][word_eachline[i]])
                    temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                    temp1.append(0)
                    temp2.append(0)
                if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                    temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]]['unk'])
                    temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                    temp1.append(0)
                    temp2.append(0)
            viterbi[j,i] = max(temp1)
            backpointer[j,i]= temp2.index(max(temp2))+1  ## here if there are more than one largest values in temp2, the index function only get the smallest index
                
    ## Temination step
    temp = []
    for i in range(1, len(tag_list)-1):
        if "</s>" in transition_matrix[tag_list[i]]:
            temp.append(viterbi[i,-1]*transition_matrix[tag_list[i]]['</s>'])
        else:
            temp.append(0)
    viterbi[tag_list.index("</s>"),-1] = max(temp)
    backpointer[tag_list.index("</s>"),-1] = temp.index(max(temp))+1
        
    ## Backtrack to get the tag sequence
    predict_eachline = ["</s>"]
    predict_eachline.append(tag_list[backpointer[-1,-1]])
    prev = backpointer[-1,-1]

    for i in range(len(word_eachline)-1, 0, -1):
        predict_eachline.append(tag_list[backpointer[prev,i]])
        prev = backpointer[prev,i]
    predict_eachline.append(tag_list[backpointer[prev,0]])
    predict_eachline = predict_eachline[::-1]
    
    ## Transform the predictd tag back to the unitag form
    predict_eachline_transform_back_unitag = ["<s>"]        
    for i in range(1,len(predict_eachline)-1):
        predict_eachline_transform_back_unitag.append(predict_eachline[i][-1])        
    predict_eachline_transform_back_unitag.append('</s>')

In [11]:
print("The prediction on the second line of test file is: {}.".format(predict_eachline_transform_back_unitag))

The prediction on the second line of test file is: ['<s>', 'F', 'F', 'D', 'D', 'F', 'F', 'D', 'D', 'D', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'F', 'F', '</s>'].


In [12]:
## Print out the 2nd line which only contains the word labeled as N 
output_label_N = []
for i in range(1,len(predict_eachline_transform_back_unitag)-1):
    if predict_eachline_transform_back_unitag[i] == 'N':
        output_label_N.append(word_eachline[i-1])
print (output_label_N)

['i', 'guess', "it's", 'pretty', 'deep', 'feelings', ',']
